In [11]:
import requests
from bs4 import BeautifulSoup as bs

html = bs(requests.get("https://movie.naver.com/movie/sdb/rank/rmovie.naver?sel=cur&date=20230302").text)
title = html.select('table.list_ranking > tbody > tr > td.title')
point = html.select('table.list_ranking > tbody > tr > td.point')

for i in range(len(title)):
    print(i + 1, title[i].text.strip(), point[i].text.strip())

1 아임 히어로 더 파이널 9.93
2 탑건: 매버릭 9.76
3 다음 소희 9.47
4 더 퍼스트 슬램덩크 9.45
5 올빼미 9.30
6 극장판 주술회전 0 9.21
7 러브레터 9.14
8 탄생 9.06
9 비긴 어게인 9.04
10 영웅 8.99
11 극장판 소드 아트 온라인 -오디널 스케일- 8.95
12 하녀 8.91
13 극장판 5등분의 신부 8.91
14 카운트 8.90
15 서치 2 8.90
16 장화신은 고양이: 끝내주는 모험 8.86
17 엘비스 8.83
18 너의 이름은. 8.81
19 여름날 우리 8.77
20 항거:유관순 이야기 8.66
21 아바타: 물의 길 8.65
22 헤어질 결심 8.64
23 라라랜드 8.63
24 오늘 밤, 세계에서 이 사랑이 사라진다 해도 8.62
25 양자경의 더 모든 날 모든 순간 8.61
26 블루 재스민 8.51
27 졸업 8.49
28 애프터썬 8.45
29 티파니에서 아침을 8.36
30 큐어 8.34
31 이장 8.33
32 상견니 8.29
33 에브리씽 에브리웨어 올 앳 원스 8.22
34 언어의 정원 8.17
35 몽상가들 8.10
36 본즈 앤 올 8.07
37 날씨의 아이 7.97
38 별을 쫓는 아이 7.89
39 바빌론 7.89
40 TAR 타르 7.78
41 사랑할 땐 누구나 최악이 된다 7.75
42 천룡팔부: 교봉전 7.39
43 메간 7.38
44 더 배트맨 7.21
45 앤트맨과 와스프: 퀀텀매니아 7.19
46 유령 6.81
47 원피스 필름 레드 6.77
48 대외비 6.27
49 블랙 팬서: 와칸다 포에버 6.04
50 마루이 비디오 5.67


In [9]:
# 스타벅스 크롤링 & 분석

from selenium import webdriver
import time
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.by import By

url = "https://www.starbucks.co.kr/store/store_map.do"
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

area_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > header.loca_search > h3 > a'
driver.find_element(By.CSS_SELECTOR, area_btn).click()

seoul_btn = '#container > div > form > fieldset > div > section > article.find_store_cont > article > article:nth-child(4) > div.loca_step1 > div.loca_step1_cont > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, seoul_btn).click()

time.sleep(1)

all_btn = '#mCSB_2_container > ul > li:nth-child(1) > a'
driver.find_element(By.CSS_SELECTOR, all_btn).click()

time.sleep(3)

txt = driver.page_source
html = bs(txt)

shops = html.select('ul.quickSearchResultBoxSidoGugun > li.quickResultLstCon')
print(len(shops))
print(shops[0])

C:\Users\82104\AppData\Local\Temp\ipykernel_13428\1162232319.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


600
<li class="quickResultLstCon" data-code="3762" data-hlytag="null" data-index="0" data-lat="37.501087" data-long="127.043069" data-name="역삼아레나빌딩" data-storecd="1509" style="background:#fff"> <strong data-my_siren_order_store_yn="N" data-name="역삼아레나빌딩" data-store="1509" data-yn="N">역삼아레나빌딩  </strong> <p class="result_details">서울특별시 강남구 언주로 425 (역삼동)<br/>1522-3232</p> <i class="pin_general">리저브 매장 2번</i></li>


In [71]:
shop = shops[0]

lat = shop.get('data-lat') # 위도
print(lat)
long = shop.get('data-long') # 경도
print(long)
name = shop.get('data-name') # 지점명
print(name)

addr, phone = str(shop.select('p')[0]).split('<br/>')
addr = addr.split('>')[1]
phone = phone.split('<')[0]
print(addr)
print(phone)

stype = shop.select('i.pin_general')[0].text # 매장종류
print(stype)

37.501087
127.043069
역삼아레나빌딩
서울특별시 강남구 언주로 425 (역삼동)
1522-3232
리저브 매장 2번
서울특별시 강남구 언주로 425 (역삼동)
1522-3232


In [86]:
import pandas as pd

shop_data = []

for shop in shops:
    lat = shop.get('data-lat') # 위도
    long = shop.get('data-long') # 경도
    name = shop.get('data-name') # 지점명

    addr, phone = str(shop.select('p')[0]).split('<br/>')
    addr = addr.split('>')[1] # 주소
    phone = phone.split('<')[0] # 전화번호

    stype = shop.select('i')[0].text # 매장종류

    shop_data.append([name, lat, long, addr, phone, stype])

df = pd.DataFrame(shop_data, columns = ['매장명', '위도', '경도', '주소', '전화번호', '매장종류'])
df
# df.to_excel('starbugs_seoul.xlsx', index=False)
# df = pd.read_excel('starbugs_seoul.xlsx', header=0)

,매장명,위도,경도,주소,전화번호,매장종류
0,역삼아레나빌딩,37.501087,127.043069,서울특별시 강남구 언주로 425 (역삼동),1522-3232,리저브 매장 2번
1,논현역사거리,37.510178,127.022223,서울특별시 강남구 강남대로 538 (논현동),1522-3232,리저브 매장 2번
2,신사역성일빌딩,37.5139309,127.0206057,서울특별시 강남구 강남대로 584 (논현동),1522-3232,리저브 매장 2번
3,국기원사거리,37.499517,127.031495,서울특별시 강남구 테헤란로 125 (역삼동),1522-3232,리저브 매장 2번
4,대치재경빌딩R,37.494668,127.062583,서울특별시 강남구 남부순환로 2947 (대치동),1522-3232,리저브 매장 2번
...,...,...,...,...,...,...
595,사가정역,37.579594,127.087966,서울특별시 중랑구 면목로 310,1522-3232,리저브 매장 2번
596,상봉역,37.59689,127.08647,서울특별시 중랑구 망우로 307 (상봉동),1522-3232,리저브 매장 2번
597,묵동,37.615368,127.076633,"서울특별시 중랑구 동일로 952 (묵동, 로프트원 태릉입구역) 1층",1522-3232,리저브 매장 2번
598,양원역,37.6066536267232,127.106359790053,서울특별시 중랑구 양원역로10길 3 (망우동),1522-3232,리저브 매장 2번


In [114]:
# 데이터 분석

# !pip install folium

In [14]:
import folium

starmap = folium.Map(
    location = [37.5666805, 126.9784147], # 구글에서 검색한 서울 시청 좌표
    zoom_start = 11,
    tiles = 'Stamen Terrain'
)

folium.Marker(
    location = [37.5666805, 126.9784147],
    fill = True
).add_to(starmap)

# starmap

In [48]:
import folium, pandas as pd

starmap = folium.Map(
    location = [37.5666805, 126.9784147], # 구글에서 검색한 서울 시청 좌표
    zoom_start = 11,
    tiles = 'Stamen Terrain'
)

df = pd.read_excel('starbugs_seoul.xlsx', header=0)

for idx in df.index:
    lat = df.loc[idx, '위도'] # 위도
    long = df.loc[idx, '경도'] # 경도

    folium.Marker(
        location = [lat, long],
        fill = True
    ).add_to(starmap)

# starmap

In [47]:
# 각 지점에서 '구'명 추출

gu_names = []

for addr in df['주소']:
    gu = addr.split()[1]
    gu_names.append(gu)

df['시군구명'] = gu_names
# df

In [46]:
# 구별 스타벅스 매장 수 계산

starbugs_count = df.pivot_table(index='시군구명', values='매장명', aggfunc='count')
# starbugs_count

In [54]:
# 거주자 수, 업체 수 등의 엑셀 파일과 결합하기

seoul_sgg = pd.read_excel('seoul_sgg_stat.xlsx', thousands=',')

seoul_sgg = pd.merge(seoul_sgg, starbugs_count, how='left', on='시군구명')
# seoul_sgg

In [55]:
# 주민등록인구 십만명당 매장수

seoul_sgg['거주자 십만명당 매장수'] = seoul_sgg['매장명'] / (seoul_sgg['주민등록인구'] / 100000)
# seoul_sgg

In [57]:
# 종사자수 십만명당 매장수

seoul_sgg['종사자수 십만명당 매장수'] = seoul_sgg['매장명'] / (seoul_sgg['종사자수_y'] / 100000)
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y,매장명,거주자 십만명당 매장수,종사자수 십만명당 매장수
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455,4,1.077891,5.825045
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179,13,2.741662,14.824444
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994,10,2.673518,6.951299
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609,11,2.696303,10.585575
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080,13,4.955458,5.828081
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445,14,3.116742,6.650642
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871,40,22.595934,14.864031
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654,6,1.750961,8.597590
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287,8,1.854931,8.061184
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590,87,15.238749,12.231504


In [61]:
# 사업체수 1000개당 매장수
seoul_sgg['사업체수_천개당_매장수'] = seoul_sgg['매장명'] / (seoul_sgg['사업체수_y'] / 1000)
# 사업체수 10000개당 매장수
seoul_sgg['사업체수_만개당_매장수'] = seoul_sgg['매장명'] / (seoul_sgg['사업체수_y'] / 10000)
seoul_sgg

,시군구코드,시군구명,위도,경도,스타벅스_매장수,주민등록인구,종사자수_x,사업체수_x,종사자수_y,사업체수_y,매장명,거주자 십만명당 매장수,종사자수 십만명당 매장수,사업체수_천개당_매장수,사업체수_만개당_매장수
0,11320,도봉구,37.665861,127.031767,2,371095,68669,18455,68669,18455,4,1.077891,5.825045,0.216743,2.167434
1,11380,은평구,37.617612,126.922700,8,474165,87693,24179,87693,24179,13,2.741662,14.824444,0.537657,5.376566
2,11230,동대문구,37.583801,127.050700,8,374039,143858,32994,143858,32994,10,2.673518,6.951299,0.303085,3.030854
3,11590,동작구,37.496504,126.944307,11,407966,103915,19609,103915,19609,11,2.696303,10.585575,0.560967,5.609669
4,11545,금천구,37.460097,126.900155,11,262337,223058,30080,223058,30080,13,4.955458,5.828081,0.432181,4.321809
5,11530,구로구,37.495486,126.858121,11,449187,210506,37445,210506,37445,14,3.116742,6.650642,0.373882,3.738817
6,11110,종로구,37.599100,126.986149,37,177023,269106,40871,269106,40871,40,22.595934,14.864031,0.978689,9.786890
7,11305,강북구,37.646995,127.014716,5,342669,69787,18654,69787,18654,6,1.750961,8.597590,0.321647,3.216468
8,11260,중랑구,37.595379,127.093967,7,431283,99241,27287,99241,27287,8,1.854931,8.061184,0.293180,2.931799
9,11680,강남구,37.495985,127.066409,81,570913,711278,73590,711278,73590,87,15.238749,12.231504,1.182226,11.822258


In [6]:
from selenium import webdriver

url = "https://www.musinsa.com/app/goods/3063926"
driver = webdriver.Chrome('chromedriver.exe')
driver.get(url)

txt = driver.page_source
html = bs(txt)

views = html.select('h4 > strong')
print(views[0].text)



C:\Users\82104\AppData\Local\Temp\ipykernel_15896\1819145981.py:4: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver.exe')


[<strong>1.3천 회 이상</strong>]
